In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

In [2]:
sys.path.append("../src")

In [3]:
from utils import read_xray
from glob import glob
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [6]:
train_local_annotations_path = "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/annotations_train.csv"
train_global_annotations_path = "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/image_labels_train.csv"

test_local_annotations_path = "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/annotations_test.csv"
test_global_annotations_path = "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/image_labels_test.csv"

In [7]:
train_local_annotations = pd.read_csv(train_local_annotations_path)
train_global_annotations = pd.read_csv(train_global_annotations_path)
test_local_annotations = pd.read_csv(test_local_annotations_path)
test_global_annotations = pd.read_csv(test_global_annotations_path)

In [16]:
print(train_global_annotations.shape)
train_global_annotations.head(10)


(45000, 30)


,image_id,rad_id,Aortic enlargement,Atelectasis,Calcification,Cardiomegaly,Clavicle fracture,Consolidation,Edema,Emphysema,...,Pneumothorax,Pulmonary fibrosis,Rib fracture,Other lesion,COPD,Lung tumor,Pneumonia,Tuberculosis,Other diseases,No finding
0,000434271f63a053c4128a0ba6352c7f,R2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,000434271f63a053c4128a0ba6352c7f,R3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,000434271f63a053c4128a0ba6352c7f,R6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,00053190460d56c53cc3e57321387478,R11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,00053190460d56c53cc3e57321387478,R2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,00053190460d56c53cc3e57321387478,R7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0005e8e3701dfb1dd93d53e2ff537b6e,R10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
7,0005e8e3701dfb1dd93d53e2ff537b6e,R8,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
8,0005e8e3701dfb1dd93d53e2ff537b6e,R9,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
9,0006e0a85696f6bb578e84fafa9a5607,R11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
print(train_local_annotations.shape)
train_local_annotations.head(10)

(69052, 7)


,image_id,rad_id,class_name,x_min,y_min,x_max,y_max
0,000434271f63a053c4128a0ba6352c7f,R2,No finding,NaN,NaN,NaN,NaN
1,000434271f63a053c4128a0ba6352c7f,R3,No finding,NaN,NaN,NaN,NaN
2,000434271f63a053c4128a0ba6352c7f,R6,No finding,NaN,NaN,NaN,NaN
3,00053190460d56c53cc3e57321387478,R11,No finding,NaN,NaN,NaN,NaN
4,00053190460d56c53cc3e57321387478,R2,No finding,NaN,NaN,NaN,NaN
5,00053190460d56c53cc3e57321387478,R7,No finding,NaN,NaN,NaN,NaN
6,0005e8e3701dfb1dd93d53e2ff537b6e,R10,Infiltration,900.956970,587.809021,1205.359985,888.710999
7,0005e8e3701dfb1dd93d53e2ff537b6e,R10,Lung Opacity,900.956970,587.809021,1205.359985,888.710999
8,0005e8e3701dfb1dd93d53e2ff537b6e,R8,Consolidation,932.471985,567.778992,1197.770020,896.408997
9,0005e8e3701dfb1dd93d53e2ff537b6e,R8,Nodule/Mass,932.471985,567.778992,1197.770020,896.408997


In [18]:
print(test_local_annotations.shape)
test_local_annotations.head(10)

(4748, 6)


,image_id,class_name,x_min,y_min,x_max,y_max
0,e0dc2e79105ad93532484e956ef8a71a,Calcification,1221.560976,661.580417,1343.298780,783.318222
1,e0dc2e79105ad93532484e956ef8a71a,Cardiomegaly,638.695122,1310.848710,1800.737805,1937.982856
2,e0dc2e79105ad93532484e956ef8a71a,ILD,1693.756098,1392.007246,1929.853659,1941.671881
3,e0dc2e79105ad93532484e956ef8a71a,Pneumothorax,399.924053,145.803282,1016.600328,510.138208
4,e0dc2e79105ad93532484e956ef8a71a,Pneumothorax,334.300846,493.762382,453.145487,744.509170
5,e0dc2e79105ad93532484e956ef8a71a,Pleural effusion,229.975703,1482.203045,644.450374,1945.701602
6,e0dc2e79105ad93532484e956ef8a71a,Atelectasis,379.275719,1248.225409,504.063792,1500.029913
7,e0dc2e79105ad93532484e956ef8a71a,Pneumothorax,283.456305,1381.926915,414.929454,1546.825440
8,0aed23e64ebdea798486056b4f174424,Pleural effusion,1593.050179,1885.493865,2164.350008,2288.954196
9,0aed23e64ebdea798486056b4f174424,Pleural effusion,162.061085,1757.177093,881.412855,2229.059285


In [19]:
print(test_global_annotations.shape)
test_global_annotations.head(10)

(3000, 29)


,image_id,Aortic enlargement,Atelectasis,Calcification,Cardiomegaly,Clavicle fracture,Consolidation,Edema,Emphysema,Enlarged PA,...,Pneumothorax,Pulmonary fibrosis,Rib fracture,Other lesion,COPD,Lung tumor,Pneumonia,Tuberculosis,Other disease,No finding
0,e0dc2e79105ad93532484e956ef8a71a,0,1,1,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
1,0aed23e64ebdea798486056b4f174424,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,aa15cfcfca7605465ca0513902738b95,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,665c4a6d2693dc0286d65ab479c9b169,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,42da2c134b53cb5594774d3d29faac59,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,c7179539654a1b3b7977e56e7e3009d5,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,bfd1974dc9778aadb407a11b57ab748f,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,618777b8305b062583337d9a6b7a3d4e,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
8,e54b5a593bc03c789ecdc18d8270964e,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,3019aec706bd013e1e3348564fbfd086,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


### Checking Data leakage

In [24]:
print(len(np.unique(train_global_annotations['image_id'])))
print(len(np.unique(test_global_annotations['image_id'])))
print(len(np.unique(np.concatenate((train_global_annotations['image_id'],test_global_annotations['image_id'])))))

15000
3000
18000


In [25]:
print(len(np.unique(train_local_annotations['image_id'])))
print(len(np.unique(test_local_annotations['image_id'])))
print(len(np.unique(np.concatenate((train_local_annotations['image_id'],test_local_annotations['image_id'])))))

15000
3000
18000


### Global annotations

In [28]:
print(np.unique(train_global_annotations['rad_id']))

['R1' 'R10' 'R11' 'R12' 'R13' 'R14' 'R15' 'R16' 'R17' 'R2' 'R3' 'R4' 'R5'
 'R6' 'R7' 'R8' 'R9']


In [36]:
col = train_global_annotations.columns.values
print(col)

['image_id' 'rad_id' 'Aortic enlargement' 'Atelectasis' 'Calcification'
 'Cardiomegaly' 'Clavicle fracture' 'Consolidation' 'Edema' 'Emphysema'
 'Enlarged PA' 'ILD' 'Infiltration' 'Lung Opacity' 'Lung cavity'
 'Lung cyst' 'Mediastinal shift' 'Nodule/Mass' 'Pleural effusion'
 'Pleural thickening' 'Pneumothorax' 'Pulmonary fibrosis' 'Rib fracture'
 'Other lesion' 'COPD' 'Lung tumor' 'Pneumonia' 'Tuberculosis'
 'Other diseases' 'No finding']


In [40]:
all_labels = np.delete(col, [0,1])
print(all_labels)
print(len(all_labels))

['Aortic enlargement' 'Atelectasis' 'Calcification' 'Cardiomegaly'
 'Clavicle fracture' 'Consolidation' 'Edema' 'Emphysema' 'Enlarged PA'
 'ILD' 'Infiltration' 'Lung Opacity' 'Lung cavity' 'Lung cyst'
 'Mediastinal shift' 'Nodule/Mass' 'Pleural effusion' 'Pleural thickening'
 'Pneumothorax' 'Pulmonary fibrosis' 'Rib fracture' 'Other lesion' 'COPD'
 'Lung tumor' 'Pneumonia' 'Tuberculosis' 'Other diseases' 'No finding']
28


In [44]:
train_global_annotations[train_global_annotations['Aortic enlargement'] == 1]

,image_id,rad_id,Aortic enlargement,Atelectasis,Calcification,Cardiomegaly,Clavicle fracture,Consolidation,Edema,Emphysema,...,Pneumothorax,Pulmonary fibrosis,Rib fracture,Other lesion,COPD,Lung tumor,Pneumonia,Tuberculosis,Other diseases,No finding
12,0007d316f756b3fa0baea2ff514ce945,R10,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
14,0007d316f756b3fa0baea2ff514ce945,R9,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
20,000d68e42b71d3eac10ccc077aba07c1,R9,1,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,1,0
21,00150343289f317a0ad5629d5b7d9ef9,R10,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
22,00150343289f317a0ad5629d5b7d9ef9,R8,1,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44963,ffb5d0b005261ed350f7a08c06613a34,R9,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
44987,ffe6f9fe648a7ec29a50feb92d6c15a4,R9,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
44991,ffeffc54594debf3716d6fcd2402a99f,R10,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
44992,ffeffc54594debf3716d6fcd2402a99f,R8,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
